In [18]:
import librosa
from panns_inference import AudioTagging, labels

# ---------- 1. Charger l'audio ----------
audio_path = "AMBNaut_Goelands sur le port (ID 2573)_LS.mp3"

# PANNs attend du 32 kHz mono
audio, sr = librosa.load(audio_path, sr=32000, mono=True)
audio = audio[None, :]  # (batch_size, n_samples)

# ---------- 2. Créer le modèle ----------
# device='cpu' pour être sûr que ça tourne partout
at = AudioTagging(checkpoint_path=None, device='cpu')

# ---------- 3. Inférence ----------
clipwise_output, embedding = at.inference(audio)

# clipwise_output: (1, 527) proba par classe AudioSet
# labels: liste de 527 noms de classes
clipwise_output = clipwise_output[0]

# On récupère les meilleurs tags
top_k = 10
indices_sorted = clipwise_output.argsort()[::-1][:top_k]

print("Top tags :")
for idx in indices_sorted:
    print(f"{labels[idx]:30s} -> {clipwise_output[idx]:.3f}")

# ---------- 4. Filtrer pour ne garder que des 'genres' musicaux ----------
# Ce n'est pas parfait, mais on peut commencer simple :
GENRE_KEYWORDS = [
    "music", "rock", "metal", "pop", "jazz", "classical",
    "hip hop", "rap", "techno", "electronic", "house",
    "trance", "reggae", "country", "funk"
]

def is_genre_like(label: str) -> bool:
    label_lower = label.lower()
    return any(kw in label_lower for kw in GENRE_KEYWORDS)

genre_scores = []
for idx in range(len(labels)):
    label = labels[idx]
    score = clipwise_output[idx]
    if score > 0.05 and is_genre_like(label):  # seuil arbitraire
        genre_scores.append((label, float(score)))

# Trier par score décroissant
genre_scores.sort(key=lambda x: x[1], reverse=True)

print("\nGenres (ou tags liés à la musique) détectés :")
for g, s in genre_scores[:10]:
    print(f"{g:30s} -> {s:.3f}")


Checkpoint path: C:\Users\romeo/panns_data/Cnn14_mAP=0.431.pth
Using CPU.
Top tags :
Speech                         -> 0.374
Owl                            -> 0.233
Goose                          -> 0.183
Honk                           -> 0.177
Bird                           -> 0.170
Animal                         -> 0.137
Fowl                           -> 0.120
Duck                           -> 0.086
Outside, rural or natural      -> 0.070
Domestic animals, pets         -> 0.067

Genres (ou tags liés à la musique) détectés :


In [ ]:
import os
from pathlib import Path
import urllib.request

#https://zenodo.org/records/3987831/files/Cnn14_mAP%3D0.431.pth

home = str(Path.home())
data_dir = os.path.join(home, "panns_data")
os.makedirs(data_dir, exist_ok=True)

url = "http://storage.googleapis.com/us_audioset/youtube_corpus/v1/csv/class_labels_indices.csv"
dest = os.path.join(data_dir, "class_labels_indices.csv")

print("Téléchargement du fichier vers :", dest)
urllib.request.urlretrieve(url, dest)
print("OK, fichier téléchargé ✅")


Téléchargement du fichier vers : C:\Users\romeo\panns_data\class_labels_indices.csv
OK, fichier téléchargé ✅


In [3]:
import librosa
from panns_inference import AudioTagging, labels

print("Nombre de labels :", len(labels))
print("Exemples de labels :", labels[:10])


Nombre de labels : 527
Exemples de labels : ['Speech', 'Male speech, man speaking', 'Female speech, woman speaking', 'Child speech, kid speaking', 'Conversation', 'Narration, monologue', 'Babbling', 'Speech synthesizer', 'Shout', 'Bellow']


In [6]:
from panns_inference import AudioTagging, labels

at = AudioTagging(checkpoint_path=None, device='cpu')
print("Modèle chargé OK !")
print("Nombre de labels :", len(labels))


Checkpoint path: C:\Users\romeo/panns_data/Cnn14_mAP=0.431.pth
Using CPU.
Modèle chargé OK !
Nombre de labels : 527
